<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Dependencies" data-toc-modified-id="Dependencies-1">Dependencies</a></span></li><li><span><a href="#Functions" data-toc-modified-id="Functions-2">Functions</a></span></li><li><span><a href="#Paths" data-toc-modified-id="Paths-3">Paths</a></span></li><li><span><a href="#Main" data-toc-modified-id="Main-4">Main</a></span></li></ul></div>

# Dependencies

# Functions

In [1]:
source("/Users/anabbi/git/ped_CapTCRseq/R/Immune_diversity.R")

# Paths

In [2]:
datapath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Data/"
plotpath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Plots/"
manifestpath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Manifests/"

gitpath <- "/Users/anabbi/git/ped_CapTCRseq/"

In [3]:
h4hpath <- "/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/"

# Main

In [4]:
load(file = paste0(datapath,"capTCRseq/TRBfiles.RData"))

In [5]:
head(file_df_dedup_trb)

,file_list,samplename,size
,<chr>,<chr>,<dbl>
264,batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt,CHP_315-05-PBMC-DNA,16011
2,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_350-01-PBMC-DNA.txt,CHP_350-01-PBMC-DNA,85631
3,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_350-02-PBMC-DNA.txt,CHP_350-02-PBMC-DNA,143632
4,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_350-03-PBMC-DNA.txt,CHP_350-03-PBMC-DNA,107190
5,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_350-04-PBMC-DNA.txt,CHP_350-04-PBMC-DNA,54188
6,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_353-02-PBMC-DNA.txt,CHP_353-02-PBMC-DNA,9062


In [6]:
onefle <- read.table(paste0(h4hpath, file_df_dedup_trb$file_list[1]), sep = "\t", header = TRUE, stringsAsFactors = F)  

In [7]:
file_df_dedup_trb$file_list[1]

[1] "batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt"

In [8]:
gliph_input <- onefle

In [9]:
mylist <- list()

for(i in 1:nrow(file_df_dedup_trb)){
    
    onefle <- read.table(paste0(h4hpath, file_df_dedup_trb$file_list[i]), 
                         sep = "\t", header = TRUE, stringsAsFactors = F)  
    # Remove non-productive
    onefle <- onefle[!grepl("_", onefle$aaSeqCDR3),]
    onefle <- onefle[!grepl("[*]", onefle$aaSeqCDR3),]
    onefle$CDR3b <- onefle$aaSeqCDR3
    #next if the file has no productive cdr3
    if(nrow(onefle) == 0){ next}
    #remove scores
    onefle$TRBV <- gsub("[*]0.*", "", onefle$allVHitsWithScore)
    onefle$TRBJ <- gsub("[*]0.*", "", onefle$allJHitsWithScore)

    onefle$count <- onefle$cloneCount
    onefle$clonefraction <- onefle$cloneFraction
    onefle$nSeqCDR3 <- onefle$nSeqCDR3
    onefle$file <- paste0(h4hpath, file_df_dedup_trb$file_list[i])
    onefle$subject.condition <- file_df_dedup_trb$samplename[i]
    mylist[length(mylist) + 1] <- list(onefle[,c("CDR3b", "TRBV", "TRBJ", "subject.condition", "count", "clonefraction",
                                                "nSeqCDR3", "file")])
}

In [10]:
length(mylist)

[1] 580

In [11]:
gliph_input <- do.call("rbind", mylist)

In [12]:
dim(gliph_input)

[1] 72634     8

In [13]:
head(gliph_input)

,CDR3b,TRBV,TRBJ,subject.condition,count,clonefraction,nSeqCDR3,file
,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<chr>,<chr>
1,CASSFYGREADTQYF,TRBV7-2,TRBJ2-3,CHP_315-05-PBMC-DNA,187,0.19767442,TGTGCCAGCAGCTTCTATGGCAGGGAAGCAGATACGCAGTATTTT,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt
3,CASSLMGGDSTEAFF,TRBV7-3,TRBJ1-1,CHP_315-05-PBMC-DNA,48,0.05073996,TGTGCCAGCAGCTTAATGGGGGGAGATAGCACTGAAGCTTTCTTT,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt
4,CASSLTGTGVEKLFF,TRBV7-3,TRBJ1-4,CHP_315-05-PBMC-DNA,48,0.05073996,TGTGCCAGCAGCTTAACCGGGACAGGGGTTGAAAAACTGTTTTTT,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt
5,CASSQRGQSTEQYF,TRBV14,TRBJ2-7,CHP_315-05-PBMC-DNA,42,0.04439746,TGTGCCAGCAGCCAACGGGGACAGTCAACCGAGCAGTACTTC,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt
6,CASSYQAGVGETQYF,TRBV6-6,TRBJ2-5,CHP_315-05-PBMC-DNA,32,0.03382664,TGTGCCAGCAGTTACCAAGCGGGGGTGGGTGAGACCCAGTACTTC,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt
9,CASSVVGVYNSPLHF,TRBV7-2,TRBJ1-6,CHP_315-05-PBMC-DNA,29,0.03065539,TGTGCCAGCAGCGTAGTAGGGGTCTATAATTCACCCCTCCACTTT,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt


In [15]:
library(data.table)

In [16]:
setDT(gliph_input)

ab <- gliph_input[, .(subject = toString(subject.condition),
                      file = toString(file),
                      count = sum(count)),  by = c("CDR3b", "TRBV")]

tm <- gliph_input[ match(ab$CDR3b, gliph_input$CDR3b),] #match back to the original dataframe

ERROR: Error in `[.data.table`(gliph_input, , `:=`(names(ab), ab)): Supplied 68956 items to be assigned to 72634 items of column 'CDR3b'. If you wish to 'recycle' the RHS please use rep() to make this intent clear to readers of your code.


In [139]:
dim(ab)

[1] 68956     5

In [17]:
dim(gliph_input)

[1] 72634     8

In [21]:
ab[ grepl(",", ab$subject),]

CDR3b,TRBV,TRBJ,subject,count
<chr>,<chr>,<chr>,<chr>,<int>
CSVTTGTEAFF,TRBV29-1,TRBJ1-1,"CHP_315-05-PBMC-DNA, CHP_315-05-PBMC-DNA",30
CASSVGGNTEAFF,TRBV9,TRBJ1-1,"CHP_315-05-PBMC-DNA, CHP_404-05-PBMC-DNA",130
CASSHRGTEAFF,TRBV3-1,TRBJ1-1,"CHP_350-01-PBMC-DNA, CHP_420-01_cfDNA",127
CASSLNTDTQYF,TRBV7-3,TRBJ2-3,"CHP_350-01-PBMC-DNA, CHP_338-05-PBMC-DNA",11
CASSDSSGSTDTQYF,TRBV6-4,TRBJ2-3,"CHP_350-02-PBMC-DNA, CHP_404-05-PBMC-DNA, CHP_368-05-PBMC-DNA, CHP_374-04-PBMC-DNA, CHP_381-02-PBMC-DNA, CHP_366-04_PBMC-DNA, CHP_363-01_PBMC-DNA, CHP_371-01_PBMC-DNA, CHP_371-04_PBMC-DNA, CHP_360-04_PBMC-DNA, CHP-371-03-PBMC-DNA",2076
CSASRAGPNEQFF,TRBV20-1,TRBJ2-1,"CHP_350-02-PBMC-DNA, CHP_350-02-PBMC-DNA",21
CASSQEEFGGGPDYEQYF,TRBV4-1,TRBJ2-7,"CHP_350-02-PBMC-DNA, CHP_350-02-PBMC-DNA",15
CASSSNTEAFF,TRBV27,TRBJ1-1,"CHP_350-02-PBMC-DNA, CHP_399-02_PBMC-DNA",185
CASSRREGQVEQFF,TRBV19,TRBJ2-1,"CHP_350-02-PBMC-DNA, CHP_350-02-PBMC-DNA",8


In [23]:
ab[ ab$CDR3b == "CSVTTGTEAFF",]

CDR3b,TRBV,TRBJ,subject,count
<chr>,<chr>,<chr>,<chr>,<int>
CSVTTGTEAFF,TRBV29-1,TRBJ1-1,"CHP_315-05-PBMC-DNA, CHP_315-05-PBMC-DNA",30


In [19]:
gliph_input$subject.condition[ !gliph_input$TRBV %in% ab$TRBV]

character(0)

In [132]:
gliph_input[, names(ab) := ab]

ERROR: Error in `[.data.table`(gliph_input, , `:=`(names(ab), ab)): Supplied 68956 items to be assigned to 72634 items of column 'CDR3b'. If you wish to 'recycle' the RHS please use rep() to make this intent clear to readers of your code.


In [46]:
dat <- gliph_input[ gliph_input$CDR3b == "CASSLGTGELFF",]

In [47]:
dat

CDR3b,TRBV,TRBJ,subject.condition,count,clonefraction,nSeqCDR3,file
<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<chr>,<chr>
CASSLGTGELFF,TRBV12-3,TRBJ2-2,CHP_391-03_PBMC_DNA,16,0.0026033192,TGTGCCAGCAGTTTAGGCACCGGGGAGCTGTTTTTT,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch11_23Apr2020/mixcr/clones/CLONES_TRBCHP_391-03_PBMC_DNA.txt
CASSLGTGELFF,TRBV11-1,TRBJ2-2,CHP_417-01_PBMC-DNA,23,0.0015535292,TGTGCCAGCAGCTTAGGGACCGGGGAGCTGTTTTTT,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch12_30Apr2021/mixcr/clones/CLONES_TRBCHP_417-01_PBMC-DNA.txt
CASSLGTGELFF,TRBV11-1,TRBJ2-2,CHP_367-04_cfDNA,35,0.0005202527,TGTGCCAGCAGCTTAGGGACCGGGGAGCTGTTTTTT,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch13_cfDNA_11Sep2020/mixcr/clones/CLONES_TRBCHP_367-04_cfDNA.txt
CASSLGTGELFF,TRBV11-1,TRBJ2-2,CHP_390-02_cfDNA,45,0.0013057482,TGTGCCAGCAGCTTAGGGACCGGGGAGCTGTTTTTT,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch13_cfDNA_11Sep2020/mixcr/clones/CLONES_TRBCHP_390-02_cfDNA.txt
CASSLGTGELFF,TRBV12-4,TRBJ2-2,CHP_365-04_PBMC-DNA,165,0.0025414331,TGTGCCAGCAGTTTAGGAACCGGGGAGCTGTTTTTT,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/remaining_batch_2_22April2021/mixcr/clones/CLONES_TRBCHP_365-04_PBMC-DNA.txt
CASSLGTGELFF,TRBV11-3,TRBJ2-2,CHP-342-03-PBMC-DNA,9,0.0004719950,TGTGCCAGCAGCTTGGGCACCGGGGAGCTGTTTTTT,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/remaining_batch_4_28June2021/mixcr/clones/CLONES_TRBCHP-342-03-PBMC-DNA.txt
CASSLGTGELFF,TRBV5-6,TRBJ2-2,CHP-386-04-PBMC-DNA,20,0.0001613840,TGTGCCAGCAGCTTGGGTACCGGGGAGCTGTTTTTT,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/remaining_batch_4_28June2021/mixcr/clones/CLONES_TRBCHP-386-04-PBMC-DNA.txt


In [51]:
#library(data.table)

setDT(dat)

ab <- dat[, .(subject = toString(subject.condition), #toString joins elements into a single column separated by ,
              file = toString(file), #join files
              TRBV = toString(TRBV), #join TRBVs (or not?)
              count = sum(count)),  by = "CDR3b"] #sum counts #collapse if CDR3 is duplicated

#dat[, names(ab) := ab]

tm <- dat[ match(ab$CDR3b, dat$CDR3b),] #match back to the original dataframe

In [52]:
tm

CDR3b       
1 CASSLGTGELFF
  TRBV                                                                TRBJ   
1 TRBV12-3, TRBV11-1, TRBV11-1, TRBV11-1, TRBV12-4, TRBV11-3, TRBV5-6 TRBJ2-2
  subject.condition   count clonefraction nSeqCDR3                            
1 CHP_391-03_PBMC_DNA 2191  0.002603319   TGTGCCAGCAGTTTAGGCACCGGGGAGCTGTTTTTT
  file                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [49]:
ab

CDR3b,subject,file,count
<chr>,<chr>,<chr>,<int>
CASSLGTGELFF,"CHP_391-03_PBMC_DNA, CHP_417-01_PBMC-DNA, CHP_367-04_cfDNA, CHP_390-02_cfDNA, CHP_365-04_PBMC-DNA, CHP-342-03-PBMC-DNA, CHP-386-04-PBMC-DNA","/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch11_23Apr2020/mixcr/clones/CLONES_TRBCHP_391-03_PBMC_DNA.txt, /Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch12_30Apr2021/mixcr/clones/CLONES_TRBCHP_417-01_PBMC-DNA.txt, /Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch13_cfDNA_11Sep2020/mixcr/clones/CLONES_TRBCHP_367-04_cfDNA.txt, /Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch13_cfDNA_11Sep2020/mixcr/clones/CLONES_TRBCHP_390-02_cfDNA.txt, /Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/remaining_batch_2_22April2021/mixcr/clones/CLONES_TRBCHP_365-04_PBMC-DNA.txt, /Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/remaining_batch_4_28June2021/mixcr/clones/CLONES_TRBCHP-342-03-PBMC-DNA.txt, /Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/remaining_batch_4_28June2021/mixcr/clones/CLONES_TRBCHP-386-04-PBMC-DNA.txt",313


In [43]:
tm <- dat[ match(ab$CDR3b, dat$CDR3b),]

In [44]:
(tm)

CDR3b,TRBV,TRBJ,subject.condition,count,clonefraction,nSeqCDR3,file,subject
<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<chr>,<chr>,<chr>
CASSLGTGELFF,TRBV12-3,TRBJ2-2,CHP_391-03_PBMC_DNA,313,0.002603319,TGTGCCAGCAGTTTAGGCACCGGGGAGCTGTTTTTT,"/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch11_23Apr2020/mixcr/clones/CLONES_TRBCHP_391-03_PBMC_DNA.txt, /Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch12_30Apr2021/mixcr/clones/CLONES_TRBCHP_417-01_PBMC-DNA.txt, /Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch13_cfDNA_11Sep2020/mixcr/clones/CLONES_TRBCHP_367-04_cfDNA.txt, /Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch13_cfDNA_11Sep2020/mixcr/clones/CLONES_TRBCHP_390-02_cfDNA.txt, /Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/remaining_batch_2_22April2021/mixcr/clones/CLONES_TRBCHP_365-04_PBMC-DNA.txt, /Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/remaining_batch_4_28June2021/mixcr/clones/CLONES_TRBCHP-342-03-PBMC-DNA.txt, /Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/remaining_batch_4_28June2021/mixcr/clones/CLONES_TRBCHP-386-04-PBMC-DNA.txt","CHP_391-03_PBMC_DNA, CHP_417-01_PBMC-DNA, CHP_367-04_cfDNA, CHP_390-02_cfDNA, CHP_365-04_PBMC-DNA, CHP-342-03-PBMC-DNA, CHP-386-04-PBMC-DNA"


In [34]:
tail(sort(table(gliph_input$CDR3b)),50)


       CASSLGTGELFF       CASSLQGNQPQHF        CASSLRGNEQFF        CASSLRGTEAFF 
                  7                   7                   7                   7 
     CASSPEGGYTEAFF         CASSPGYGYTF        CASSPPGYGYTF   CASSQDSQGGPNYGYTF 
                  7                   7                   7                   7 
    CASSQGTEGNQPQHF CASSRNTQPQVGRSNEQFF      CASSVGGTGNTIYF       CASSVGTRYEQYF 
                  7                   7                   7                   7 
        CATELDYEQYF      CSARDRGLNTEAFF      CSARGQEITEKLFF      CSRVGPEGTDTQYF 
                  7                   7                   7                   7 
 CSVGKTGRANRLNTEAFF       CASSDSNTGELFF        CASSLGGYGYTF       CASSLGPNTEAFF 
                  7                   8                   8                   8 
       CASSLGTDTQYF         CASSLGYEQYF       CASSLTGNTEAFF      CASSPLNSNQPQHF 
                  8                   8                   8                   8 
    CASSQGLAGSYEQYF   CASSR

In [74]:
head(dat2)

CDR3b,V5
<chr>,<chr>
CANHGGSEQYV,CHP_391-02_PBMC_DNA
CASEPPGSVNEQFF,CHP_350-04-PBMC-DNA
CASETDGGGTDTQYF,CHP_387-02_PBMC_DNA
CASIRGGDKEKLFF,CHP_367-01-PBMC-DNA
CASREDSYQETQYF,CHP_392-02_PBMC_DNA
CASRNRGRYGYTF,CHP_352-02-PBMC-DNA


In [67]:
?group_by_at

In [ ]:
# Remove scores from VJ columns

    


In [ ]:



for(f in file_list_ds){
    filename <- gsub(".*CLONES_", "CLONES_", f)
    filename <- gsub(".txt", "", filename)

    mixcrfle <- read.table(paste0(datapath, f),sep = "\t", header = TRUE, stringsAsFactors = F)  

    gliph_input <- mixcrfle[,c("aaSeqCDR3", "allVHitsWithScore",
                               "allJHitsWithScore", "cloneCount")]
    colnames(gliph_input) <- c("CDR3b", "TRBV", "TRBJ", "Counts")
# Remove scores from VJ columns
    gliph_input$TRBV <- gsub("[*]0.*", "", gliph_input$TRBV)
    gliph_input$TRBJ <- gsub("[*]0.*", "", gliph_input$TRBJ)
# Remove non-productive
    gliph_input <- gliph_input[!grepl("_", gliph_input$CDR3b),]
    gliph_input <- gliph_input[!grepl("[*]", gliph_input$CDR3b),]
# Remove singlet and doublets    
    gliph_input <- gliph_input[gliph_input$Counts > 2,]
    
    write.table(gliph_input, file = paste0(outpath, filename, ".GLIPHinput.txt"),
                sep = "\t", row.names = FALSE, quote = FALSE)
}


In [49]:
filelist <- list.files(datapath, pattern = "ove10")

In [51]:
all.files <- lapply(filelist, function(f){ read.table(paste0(datapath,f), sep = "\t", header = FALSE,
                                                           stringsAsFactors = F)})

In [54]:
filenames <- gsub("CLONES_TRB", "", filelist)
filenames <- gsub("_20000.*", "", filenames)

In [56]:
filelst <- mapply(cbind, all.files, "filename"=filenames, SIMPLIFY=F)

In [83]:
gliph.df <- as.data.frame(do.call(rbind, filelst))

In [84]:
gliph.df$filename <- as.character(gliph.df$filename)

In [85]:
head(gliph.df)

V1,V2,V3,V4,V5,V6,filename
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Motif,Counts,avgRef,topRef,OvE,p-value,023-Blood
Motif,Counts,avgRef,topRef,OvE,p-value,CHP_303-05-PBMC-DNA
LGKE,3,0.01,1,249.999,0.001,CHP_303-05-PBMC-DNA
LFFG,3,0,1,3000,0.001,CHP_303-05-PBMC-DNA
PTLR,3,0.01,1,176.47,0.001,CHP_303-05-PBMC-DNA
RLTV,3,0.01,1,157.894,0.001,CHP_303-05-PBMC-DNA


In [86]:
colnames(gliph.df) <- c("Motif", "Counts", "avgRef", "topRef", "OvE", "p-value","samplename")

In [87]:
gliph.df <- gliph.df[gliph.df$Motif != "Motif", ]

In [88]:
gliph.df$Counts <- as.numeric(gliph.df$Counts)

In [89]:
summary(gliph.df$Counts)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  3.000   3.000   3.000   3.511   4.000   8.000 

In [90]:
table(gliph.df$samplename)


  CHP_303-05-PBMC-DNA   CHP_323-05-PBMC-DNA   CHP_331-05-PBMC-DNA 
                   32                     6                     1 
  CHP_335-03-PBMC-DNA   CHP_338-03-PBMC-DNA   CHP_339-05-PBMC-DNA 
                    1                    22                     3 
  CHP_340-05-PBMC-DNA   CHP_345-03-PBMC-DNA   CHP_347-01-PBMC-DNA 
                    5                     1                     1 
  CHP_347-02-PBMC-DNA   CHP_348-01-PBMC-DNA   CHP_350-02-PBMC-DNA 
                    2                     1                     1 
  CHP_353-01-PBMC-DNA   CHP_353-05-PBMC-DNA   CHP_357-01-PBMC-DNA 
                    1                     2                     1 
  CHP_359-02-PBMC-DNA   CHP_361-01-PBMC-DNA   CHP_362-02-PBMC-DNA 
                    1                     1                     3 
  CHP_364-01-PBMC-DNA   CHP_366-01-PBMC-DNA   CHP_368-04-PBMC-DNA 
                    1                     1                     1 
  CHP_368-05-PBMC-DNA   CHP_370-05-PBMC-DNA      CHP_371-01-c

In [73]:
gliph.df[order(gliph.df$Counts, decreasing = T),]

,Motif,Counts,avgRef,topRef,OvE,p-value,samplename
,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<fct>
110,LRG,8,0.74,4,10.738,0.001,CHP_347-01-PBMC-DNA
347,TNE,7,0.54,4,12.891,0.001,LFS_TCR-4837-PBMC-DNA
348,ATNE,7,0.18,2,38.674,0.001,LFS_TCR-4837-PBMC-DNA
349,ATN,7,0.24,3,29.045,0.001,LFS_TCR-4837-PBMC-DNA
350,TNEK,7,0.42,4,16.509,0.001,LFS_TCR-4837-PBMC-DNA
11,VLG,6,0.53,4,11.214,0.001,CHP_303-05-PBMC-DNA
22,GEL,5,0.21,3,22.935,0.001,CHP_303-05-PBMC-DNA
61,RMNT,5,0.33,4,14.97,0.001,CHP_338-03-PBMC-DNA
72,RMN,5,0.37,4,13.192,0.001,CHP_338-03-PBMC-DNA


In [92]:
gliph.df[grepl("TNE",gliph.df$Motif), ]

,Motif,Counts,avgRef,topRef,OvE,p-value,samplename
,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
344,TNE,4,0.33,3,11.904,0.001,LFS_TCR-4759-PBMC-DNA
347,TNE,7,0.54,4,12.891,0.001,LFS_TCR-4837-PBMC-DNA
348,ATNE,7,0.18,2,38.674,0.001,LFS_TCR-4837-PBMC-DNA
350,TNEK,7,0.42,4,16.509,0.001,LFS_TCR-4837-PBMC-DNA


In [78]:
gliph.df[gliph.df$Motif == "FGEG", ]

,Motif,Counts,avgRef,topRef,OvE,p-value,samplename
,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<fct>
27,FGEG,3,0.01,1,249.999,0.001,CHP_303-05-PBMC-DNA
71,FGEG,3,0.01,1,157.894,0.001,CHP_338-03-PBMC-DNA
226,FGEG,3,0.01,1,214.285,0.001,CHP_368-05-PBMC-DNA
281,FGEG,3,0.01,1,157.894,0.001,CHP_376-03-PBMC-DNA
307,FGEG,3,0.02,1,119.999,0.001,CHP_378-01-PBMC-DNA


In [94]:
gliph.df[gliph.df$Motif == "CPT", ]

,Motif,Counts,avgRef,topRef,OvE,p-value,samplename
,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
21,CPT,3,0,1,428.571,0.001,CHP_303-05-PBMC-DNA
69,CPT,3,0.02,1,115.384,0.001,CHP_338-03-PBMC-DNA
222,CPT,3,0.01,2,199.999,0.001,CHP_368-05-PBMC-DNA
276,CPT,3,0.01,2,199.999,0.001,CHP_376-03-PBMC-DNA
303,CPT,3,0.02,1,142.857,0.001,CHP_378-01-PBMC-DNA


In [74]:
table(gliph.df$Motif)


AEVS AGGN AGWA  APE APES  ATN ATNE AVGA  AVL AVLR   CP  CPT  EAV  EFQ EGSR  ELF 
   3    1    4    2    5    1    1    1    1    4    1    5    1    1    1    1 
 ERA ESSE  EVS EVSS FEGA   FF  FFG FFGE FGEG FLTG  FQF  GAP GAPE  GEL GELF GKEA 
   1    5    1    1    1    1    1    4    5    1    1    1    1    1    3    5 
GLYL  GQN  GRR  GSI GSRL  GVV  GWA GWAE KEAV  LAR   LF  LFF LFFG  LGK LGKE  LGR 
   1    1    1    1    2    1    1    1    5    1    1    5    4    1    5    1 
LGTE  LRG LRGA  LRT LRTD  LTV LTVL   NF  PES PESS  PNE  PTL PTLR   QF  QFS QFSS 
   1    1    2    1    1    1    5    1    1    1    1    1    2    1    1    1 
 QGN RAGW RGAV RLTV  RMN RMNT  RRG SAGG  SCP  SDS SEFQ SEGQ  SFE SFLT SLGR  SPY 
   1    1    1    2    1    1    1    1    5    1    1    1    1    1    1    1 
 SSC SSER TGNT TGTD  TGW  TLR TLRG  TNE TNEK TNYG  TVL TVLG VGAP  VLG VLGK  VLR 
   5    1    2    1    1    3    1    2    1    1    1    5    3    1    6    3 
VLRG  VSS  YTG 
   2    1  

In [99]:
gliph.df[grepl("368",gliph.df$samplename),]

,Motif,Counts,avgRef,topRef,OvE,p-value,samplename
,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
201,LF,4,0.31,3,12.861,0.001,CHP_368-04-PBMC-DNA
204,LGKE,3,0.01,1,166.666,0.001,CHP_368-05-PBMC-DNA
205,AVLR,3,0,1,750,0.001,CHP_368-05-PBMC-DNA
206,VLR,4,0.18,3,21.857,0.001,CHP_368-05-PBMC-DNA
207,PTLR,3,0.02,1,103.448,0.001,CHP_368-05-PBMC-DNA
208,GWA,4,0.38,3,10.526,0.001,CHP_368-05-PBMC-DNA
209,VLRG,4,0.05,2,71.428,0.001,CHP_368-05-PBMC-DNA
210,APES,3,0,1,500,0.001,CHP_368-05-PBMC-DNA
211,LGK,4,0.18,3,22.222,0.001,CHP_368-05-PBMC-DNA
